In [17]:
# USER OPTIONS
# define tokenizer name - should be one among the keys in the cell below
# tokenizer_name = 'ChordSymbolTokenizer' # or any other name from the keys in tokenizers dictionary
# tokenizer_name = 'RootTypeTokenizer'
tokenizer_name = 'PitchClassTokenizer'
# folder to xmls
val_dir = '/media/maindisk/maximos/data/gjt_melodies/Library_melodies'
# val_dir = '/media/datadisk/datasets/gjt_melodies/Library_melodies'
# val_dir = '/media/maximos/9C33-6BBD/data/gjt_melodies/Library_melodies'

# define batch size depending on GPU availability / status
batchsize = 16
# select device name - could be 'cpu', 'cuda', 'coda:0', 'cuda:1'...
device_name = 'cpu'

In [18]:
from data_utils import MergedMelHarmDataset, PureGenCollator
import os
import numpy as np
from harmony_tokenizers_m21 import ChordSymbolTokenizer, RootTypeTokenizer, \
    PitchClassTokenizer, RootPCTokenizer, GCTRootPCTokenizer, \
    GCTSymbolTokenizer, GCTRootTypeTokenizer, MelodyPitchTokenizer, \
    MergedMelHarmTokenizer
from torch.utils.data import DataLoader
from transformers import RobertaConfig, RobertaForMaskedLM, AutoConfig, GPT2LMHeadModel
import torch
from tqdm import tqdm

In [19]:
tokenizers = {
    'ChordSymbolTokenizer': ChordSymbolTokenizer,
    'RootTypeTokenizer': RootTypeTokenizer,
    'PitchClassTokenizer': PitchClassTokenizer,
    'RootPCTokenizer': RootPCTokenizer,
    'GCTRootPCTokenizer': GCTRootPCTokenizer,
    'GCTSymbolTokenizer': GCTSymbolTokenizer,
    'GCTRootTypeTokenizer': GCTRootTypeTokenizer
}

In [20]:
melody_tokenizer = MelodyPitchTokenizer.from_pretrained('saved_tokenizers/MelodyPitchTokenizer')
harmony_tokenizer = tokenizers[tokenizer_name].from_pretrained('saved_tokenizers/' + tokenizer_name)

tokenizer = MergedMelHarmTokenizer(melody_tokenizer, harmony_tokenizer)

In [21]:
collator = PureGenCollator(tokenizer)
val_dataset = MergedMelHarmDataset(val_dir, tokenizer, max_length=None, return_harmonization_labels=True)
model_path = 'saved_models/gen/' + tokenizer_name + '/' + tokenizer_name + '.pt'

valloader = DataLoader(val_dataset, batch_size=batchsize, shuffle=True, collate_fn=collator)

In [22]:
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer.vocab),
    n_positions=2048,
    n_layer=4,
    n_head=4,
    pad_token_id=tokenizer.vocab[tokenizer.pad_token],
    bos_token_id=tokenizer.vocab[tokenizer.bos_token],
    eos_token_id=tokenizer.vocab[tokenizer.eos_token],
    n_embd=256
)

model = GPT2LMHeadModel(config)

checkpoint = torch.load(model_path, map_location="cpu", weights_only=True)
model.load_state_dict(checkpoint)

model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(209, 256)
    (wpe): Embedding(2048, 256)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-3): 4 x GPT2Block(
        (ln_1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=768, nx=256)
          (c_proj): Conv1D(nf=256, nx=256)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=1024, nx=256)
          (c_proj): Conv1D(nf=256, nx=1024)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=256, out_features=209, bias=False)
)

In [23]:
if device_name == 'cpu':
    device = torch.device('cpu')
else:
    if torch.cuda.is_available():
        device = torch.device(device_name)
    else:
        print('Selected device not available: ' + device_name)
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(209, 256)
    (wpe): Embedding(2048, 256)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-3): 4 x GPT2Block(
        (ln_1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=768, nx=256)
          (c_proj): Conv1D(nf=256, nx=256)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=1024, nx=256)
          (c_proj): Conv1D(nf=256, nx=1024)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=256, out_features=209, bias=False)
)

In [24]:
with torch.no_grad():
    with tqdm(valloader, unit='batch') as tepoch:
        tepoch.set_description(f'Running')
        print(tepoch)
        for batch in tepoch:
            input_tokens = []
            real_tokens = []
            output_tokens = []
            for b in batch['input_ids']:
                # find the start harmony token
                start_harmony_position = np.where( b == tokenizer.vocab[tokenizer.harmony_tokenizer.start_harmony_token] )[0][0]
                real_ids = b
                input_ids = b[:(start_harmony_position+1)].to(device)
                for i in input_ids:
                    input_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
                print('input_tokens: ', input_tokens)

                for i in real_ids:
                    real_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
                print('real_tokens: ', real_tokens)
                
                outputs = model.generate(input_ids=input_ids.reshape(1, input_ids.shape[0]), eos_token_id=tokenizer.eos_token_id, max_new_tokens=500)
                for i in outputs[0]:
                    output_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
                print('output_tokens: ', output_tokens)
                break
            break

Running:   0%|          | 0/41 [00:00<?, ?batch/s]

Running:   0%|          | 0/41 [00:00<?, ?batch/s]
input_tokens:  ['<s>', '<bar>', 'ts_4x4', 'position_0x00', 'P:67', 'position_2x00', 'P:71', '<bar>', 'position_0x00', 'P:69', 'position_1x00', 'P:67', 'position_2x00', 'P:64', 'position_3x00', 'P:62', '<bar>', 'position_0x00', 'P:64', '<bar>', 'position_0x00', 'P:64', 'position_1x00', 'P:67', 'position_2x00', 'P:64', 'position_3x00', 'P:62', '<bar>', 'position_0x00', 'P:64', 'position_1x00', 'P:67', 'position_2x00', 'P:64', 'position_3x00', 'P:62', '<bar>', 'position_0x00', 'P:64', 'position_1x00', 'P:72', 'position_3x00', 'P:64', '<bar>', 'position_0x00', 'P:67', '<bar>', 'position_0x00', 'P:67', 'position_2x00', 'P:69', 'position_3x00', 'P:71', '<bar>', 'position_0x00', 'P:72', 'position_2x00', 'P:72', '<bar>', 'position_0x00', 'P:72', 'position_1x00', 'P:74', 'position_3x00', 'P:71', '<bar>', 'position_0x00', 'P:69', 'position_2x00', 'P:69', '<bar>', 'position_0x00', 'P:67', 'position_2x00', 'P:64', 'position_3x00', 'P:66', '<bar>',

Running:   0%|          | 0/41 [00:02<?, ?batch/s]

output_tokens:  ['<s>', '<bar>', 'ts_4x4', 'position_0x00', 'P:67', 'position_2x00', 'P:71', '<bar>', 'position_0x00', 'P:69', 'position_1x00', 'P:67', 'position_2x00', 'P:64', 'position_3x00', 'P:62', '<bar>', 'position_0x00', 'P:64', '<bar>', 'position_0x00', 'P:64', 'position_1x00', 'P:67', 'position_2x00', 'P:64', 'position_3x00', 'P:62', '<bar>', 'position_0x00', 'P:64', 'position_1x00', 'P:67', 'position_2x00', 'P:64', 'position_3x00', 'P:62', '<bar>', 'position_0x00', 'P:64', 'position_1x00', 'P:72', 'position_3x00', 'P:64', '<bar>', 'position_0x00', 'P:67', '<bar>', 'position_0x00', 'P:67', 'position_2x00', 'P:69', 'position_3x00', 'P:71', '<bar>', 'position_0x00', 'P:72', 'position_2x00', 'P:72', '<bar>', 'position_0x00', 'P:72', 'position_1x00', 'P:74', 'position_3x00', 'P:71', '<bar>', 'position_0x00', 'P:69', 'position_2x00', 'P:69', '<bar>', 'position_0x00', 'P:67', 'position_2x00', 'P:64', 'position_3x00', 'P:66', '<bar>', 'position_0x00', 'P:67', 'position_2x00', 'P:67',